In [203]:
import numpy as np
import pandas as pd
import time
from datetime import datetime
import calendar









In [204]:
def date_time_index(df):
    df['datetime'] = pd.to_datetime(df['Datum']+df['Zeit'],format= "%d-%m-%y%H:%M:%S")
    df.drop(['Datum','Zeit'],inplace=True,axis=1)
    df['Wert'] = df['Wert'].str.replace(',', '.')
    df=df.set_index('datetime')
    return df

In [205]:
def merge_wind(lis,name):
    
    power = pd.read_csv('{}.CSV'.format(lis[0]),sep=';',header = 4)
    power.drop('Status',inplace=True,axis=1)
    power_df = date_time_index(power)
    
    
    wind_speed = pd.read_csv('{}.CSV'.format(lis[1]),sep=';',header = 4)
    wind_speed.drop('Status',inplace=True,axis=1)
    wind_speed_df = date_time_index(wind_speed)
    
    
    wind_direction = pd.read_csv('{}.CSV'.format(lis[2]),sep=';',header = 4)
    wind_direction.drop('Status',inplace=True,axis=1)
    wind_direction_df = date_time_index(wind_direction)
    
    
    temperature = pd.read_csv('{}.CSV'.format(lis[3]),sep=';',header = 4)
    temperature.drop('Status',inplace=True,axis=1)
    temperature_df = date_time_index(temperature)
    temperature_df = temperature_df.resample('15T').sum().transform(lambda x: x.replace(to_replace=0, method='ffill'))

    merge_df = power_df.merge(wind_speed_df,left_index=True, right_index=True)
    merge_df = merge_df.merge(wind_direction_df,left_index=True, right_index=True)
    merge_df = merge_df.merge(temperature_df,left_index=True, right_index=True)

    merge_df.columns = ['power','wind_speed','wind_direction','temperature']
    
    merge_df.to_csv('{}.csv'.format(name),sep=',')


In [206]:
merge_wind(['KWMGW1E1PW','KWMGW1K1PW','KWMGW1K2PW','GWAT_BHHPW'],'Meckel_1')
merge_wind(['KWMGW4E1PW','KWMGW4K1PW','KWMGW4K2PW','GWAT_BHHPW'],'Meckel_4')
merge_wind(['KWRFW3E1PW','KWRFW3K1PW','KWRFW3K2PW','KLHERKATPW'],'Reinsfeld_3')